# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании. Нужно разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию, и чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('/datasets/insurance.csv')
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
X = df.drop('Страховые выплаты', axis=1)
y = df['Страховые выплаты']

## Умножение матриц

Обратимая матрица - это матрица, для которой существует обратная матрица. Для неквадратных матриц и вырожденных матриц обратных матриц не существует. Поэтому для умножения матрицы признаков на произвольную обратимую матрицу, нужно чтобы размер такой матрицы был равен $n*n$, где n - это кол-во признаков (в нашем случае 4 на 4)

Докажем, что при умножении матрицы признаков на обратимую матрицу качество линейной регрессии не изменится

Запишем все объекты в матрицу $X$ (каждая строка отвечает одному объекту). Пусть наша модель линейной регрессии задана столбцом $w$. Тогда предсказания нашей модели это столбец $Xw$ (на каждой строке – предсказание для соответствующего объекта).

Домножим $X$ на квадратную обратимую матрицу $A$. Возьмём модель линейной регрессии, заданную столбцом $A^{-1}w$. Тогда эта модель на наших объектах будет давать предсказания $(XA)(A^{-1}w)=Xw$

Тем самым, после умножения $X$ на квадратную обратимую матрицу $A$ мы всё равно можем построить модель линейной регрессии, которая даёт те же результаты, что и до умножения.

## Алгоритм преобразования

**Алгоритм**
* Создадим квадратную матрицу P размером n x n, где n - количество признаков, со случайными значениями
* Проверим, что существует обратная ей матрица.
* Умножим датасет с признаками размером m x n (где m - количество объектов) на созданную матрицу и получаем новую матрицу признаков, которая также имеет размер m x n.

In [5]:
def transform(X, random_state=None):
    
    if random_state:
        np.random.seed(random_state)
    n = X.shape[1]
    P = np.random.normal(size=(n, n))
    
    try:
        P_inv = np.linalg.inv(P)
        return X.values.dot(P)
    except:
        print('Матрица необратима. Попробуйте другой random_state')

In [6]:
# Функция для обратного шифрования, чтобы не терять данные

def transform_back(X_new, random_state):
    np.random.seed(random_state)
    n = X_new.shape[1]
    P = np.random.normal(size=(n, n))
    P_inv = np.linalg.inv(Z)
    return np.dot(X_new, P_inv)

## Проверка алгоритма

Для начала определим метрику до преобразований

In [7]:
model = LinearRegression()

In [8]:
cross_val_score(model, X, y, cv=10).mean()

0.414600847386013

Проверим наш алгоритм для десяти различных random_seed

In [9]:
for i in range(10):
    X_new = transform(X, i)
    print(cross_val_score(model, X_new, y, cv=10).mean())

0.4146008473859516
0.4146008473856896
0.4146008473860661
0.41460084738607855
0.41460084738601904
0.4146008473859745
0.41460084738599806
0.41460084738607456
0.41460084738597747
0.414600847385478


Качество модели не изменилось) Алгоритм прошел проверку!